In [9]:
from lxml import etree
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [10]:
#Change the url as you want！！！
url = 'https://steamcommunity.com/app/1948980/reviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=all'
#UA 
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'
}
#Selenium仿真操作
bro = webdriver.Chrome()
bro.get(url)

In [11]:
#每次循环获取7个评论！！！300是我调过的，之前是500，根据评论多少来调整
#现在Steam拉到最下面有一定的几率评论不会持续出现，需要往上滚一下，然后再往下移动一下，才能持续出现……
for i in range(0,300):
    bro.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    sleep(2)

In [12]:
#每次循环获取7个评论！！！
page_text = bro.page_source
tree = etree.HTML(page_text)
div_list = tree.xpath('//*[@id="AppHubCards"]/div')
values = []
happy_nums = []
suggests = []
hours = []
dates = []
comments = []
for di in div_list:
    lis = di.xpath('./div')
    for div in lis:
        value = div.xpath('.//div[@class="found_helpful"]/text()')[0]
        value = value.strip()
        value = filter(str.isdigit, value)
        value = (''.join(list(value)))
        values.append(value)
        happy = div.xpath('.//div[@class="found_helpful"]/text()')[1]
        happy = happy.strip()
        happy = filter(str.isdigit, happy)
        happy = (''.join(list(happy)))
        happy_nums.append(happy)
        suggest= div.xpath('.//div[@class="title"]//text()')[0]
        suggests.append(suggest)
        hour= div.xpath('.//div[@class="hours"]//text()')[0]
        hours.append(hour)
        date= div.xpath('.//div[@class="date_posted"]/text()')[0]
        dates.append(date)
        try:         
            comment= div.xpath('.//div[@class="apphub_UserReviewCardContent"]/div[3]/text()')[2]
            comment = comment.strip()
            comments.append(comment)
        except Exception as e:
            comment= div.xpath('.//div[@class="apphub_UserReviewCardContent"]/div[3]/text()')
            comments.append(comment)
dic = {
    'People think it\'s useful':values,
    'People think it\'s humorous':happy_nums,
    'Suggest':suggests,
    'Play Time':hours,
    'Date':dates,
    'Comments':comments
}
df = pd.DataFrame(dic)
df.to_excel('./SteamComments_daxializhizhuan.xls',encoding="utf_8_sig")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21908\1240312962.py:46: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('./SteamComments_daxializhizhuan.xls',encoding="utf_8_sig")
